In [1]:
import pandas as pd
import pdfplumber
import os
import re

In [ ]:
# Caminho do PDF
pdf_path = r'\meds_pipeline\data\0_raw\BRASINDICE_PF.pdf'
# Lista para armazenar os dados
dados = []

In [12]:
# Função para extrair dados do PDF seguindo lógica de layout
def extrair_dados_linha(linha):
    try:
        # Captura o Código TISS (10 dígitos) e o Código TUSS (até 8 dígitos), separados por espaços
        match = re.match(r'^(\d{10})\s+(\d{0,8})\s+(.*)', linha)
        if not match:
            return None

        codigo_tiss = match.group(1).strip()
        codigo_tuss = match.group(2).strip()
        restante = match.group(3).strip()

        # Encontrar o valor: último número com vírgula, seguido ou não de "REAL"
        match_valor = re.search(r'(\d{1,3}(?:\.\d{3})*,\d{2,4})\s*(REAL)?$', restante)
        if match_valor:
            valor = match_valor.group(1).replace('.', '').replace(',', '.')
            cobranca = match_valor.group(2) if match_valor.group(2) else ''
            texto_antes_valor = restante[:match_valor.start()].rstrip()

            # Laboratório: última palavra antes do valor
            partes = texto_antes_valor.rsplit(' ', 1)
            if len(partes) == 2:
                produto = partes[0].strip()
                laboratorio = partes[1].strip()
            else:
                produto = texto_antes_valor
                laboratorio = ''

            return [codigo_tiss, codigo_tuss, produto, laboratorio, valor, cobranca]
        else:
            return None
    except Exception as e:
        return None

In [13]:
# Percorre o arquivo PDF e aplica a função para extrair texto completo da página

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages[:288]:  # quantidade de paginas
        linhas = page.extract_text().split('\n')
        for linha in linhas:
            if linha.strip() and re.match(r'^\d{10}', linha):  # Linha válida começa com 10 dígitos
                dados_linha = extrair_dados_linha(linha)
                if dados_linha:
                    dados.append(dados_linha)

In [14]:
# Criar DataFrame
colunas = ['codigo_tiss', 'codigo_tuss', 'produto', 'laboratorio', 'valor', 'cobranca']
df = pd.DataFrame(dados, columns=colunas)

In [15]:
print(df.sample(10))

      codigo_tiss codigo_tuss  \
6096   0000041973    90045912   
5191   0000000667    90128524   
12643  0000055582    90294521   
11007  0000004441    90150228   
10743  0000073442    90397673   
9991   0000013674    90009606   
11213  0000071075    90383516   
2      0000000010    90269470   
13306  0000063638    90339720   
14549  0000065761    90351100   

                                                 produto     laboratorio  \
6096   DORMIRE (RESTRITO HOSP.) 2 MG/ML - CX. 12 FRS....       CRISTALIA   
5191              DDAVP 0,1 MG/ML SPRAY NASAL FR. 2,5 ML         FERRING   
12643  PREGABALINA - GENERICO 75 MG. BL. 30 CAPS. GEL...    BIOSINTETICA   
11007                              NICORD 5 MG. 20 CPRS.          MARJAN   
10743  NAROPIN (RESTRITO HOSP.) 10 MG/ML. SOL. INJ. C...          PHARMA   
9991                            MECLIN 50 MG. X 15 CPRS.           APSEN   
11213  NORFLOXACINO - GENERICO 400 MG. 2 BL. X 7 CPRS...         QUIMICA   
2                           

In [16]:
df['valor'] = df['valor'].astype(float)

In [ ]:
df.to_csv(r'C:\meds_pipeline\data\1_trusted\tabala_brasindice.csv',index=False)